# 3. Generalized Additive Models (GAM) 
#http://archive.ics.uci.edu/ml/machine-learning-databases/spambase/


In [2]:
## read the data from given URL
import pandas as pd
import numpy as np

url = "http://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data"
spam = pd.read_csv(url, header=None)
spam.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [3]:
print(np.shape(spam))

(4601, 58)


In [4]:
#set seed to make sure the sample result fixed
import random
random.seed(10)
flag = np.sort(random.sample([i for i in range(0, 4601)], 1536))
# flag

In [5]:
spamtest = spam.loc[flag, :]
spamtrain = spam.drop(np.reshape(flag,1536, 1)) #Drop a row by index

In [6]:
len(spamtest[spamtest.loc[:, 57] == 1])

609

In [7]:
data_train ={}
data_train['y'] = spamtrain.loc[:, 57]
data_train['x'] = spamtrain.loc[:, 0:56]

In [8]:
# fit using glm package
import statsmodels.api as sm
import statsmodels.formula.api as smf
 
mylogit = smf.glm(formula='y ~ x', data=data_train, family=sm.families.Binomial())
res = mylogit.fit()
print(res.summary())

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))


                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                 3065
Model:                            GLM   Df Residuals:                     3007
Model Family:                Binomial   Df Model:                           57
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Mon, 06 Jan 2020   Deviance:                          nan
Time:                        00:55:49   Pearson chi2:                 4.34e+04
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.6206      0.177     -9.148      0.0

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:942: RuntimeWarning: divide by zero encountered in true_divide
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:943: RuntimeWarning: divide by zero encountered in log
  n * np.log(1 - mu)) * var_weights
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:943: RuntimeWarning: invalid value encountered in add
  n * np.log(1 - mu)) * var_weights


In [9]:
res.resid_response

0        4.333082e-01
1        4.919134e-03
3        2.854382e-01
5        3.434233e-01
6        2.552797e-01
7        3.424036e-01
8        4.613659e-05
9        9.643175e-02
10       2.204128e-01
11       4.647918e-01
13       2.975451e-01
15       1.237023e-02
16       4.892743e-01
18       2.886331e-01
19       1.198591e-05
21       2.672188e-02
22       3.480996e-02
24       7.463614e-01
25       2.675756e-02
26       7.463614e-01
27       9.656920e-05
30       6.726482e-01
32       1.750464e-01
34       9.798602e-01
35       5.506406e-01
36       3.578743e-02
37       6.088182e-03
40       9.947064e-03
42       3.517664e-02
43       3.036354e-01
            ...      
4559    -2.034514e-01
4562    -2.013762e-28
4563    -2.760363e-02
4564    -2.289693e-01
4565    -2.788900e-02
4567    -3.366338e-12
4568    -2.253144e-04
4569    -1.720389e-01
4570    -3.901046e-02
4572    -2.647525e-02
4574    -1.750400e-01
4575    -3.323130e-03
4577    -7.458528e-02
4578    -1.555462e-01
4579    -4

In [10]:
## predict the test data based on the fitted model
# predictions = res.predict(), cannot take x_test in
predictions=np.dot(sm.add_constant(spamtest.loc[:, 0:56]),res.params)

C:\Users\tianh\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [11]:
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]

In [12]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(data_test['y'], 
                       predictions_nominal))

NameError: name 'data_test' is not defined

In [13]:
test_err = (31+97)/1536
test_err

0.08333333333333333

In [14]:
print(res.pvalues)

Intercept    5.819129e-20
x[0]         4.170484e-01
x[1]         1.319479e-01
x[2]         6.434782e-01
x[3]         2.047844e-01
x[4]         7.941578e-06
x[5]         8.029455e-05
x[6]         1.257028e-08
x[7]         7.188677e-03
x[8]         1.769652e-01
x[9]         1.177570e-01
x[10]        9.270499e-01
x[11]        5.096216e-03
x[12]        2.447498e-01
x[13]        2.174776e-01
x[14]        1.090707e-02
x[15]        3.216358e-09
x[16]        4.665732e-04
x[17]        2.952054e-01
x[18]        1.260646e-01
x[19]        3.945693e-02
x[20]        9.234293e-03
x[21]        3.332791e-01
x[22]        1.680471e-05
x[23]        1.231833e-05
x[24]        3.429889e-06
x[25]        6.756055e-03
x[26]        4.482160e-04
x[27]        3.618653e-02
x[28]        1.963581e-01
x[29]        1.047769e-01
x[30]        7.703868e-01
x[31]        8.699306e-01
x[32]        1.740485e-01
x[33]        7.449940e-03
x[34]        4.898515e-02
x[35]        7.030925e-03
x[36]        7.304500e-01
x[37]       

In [15]:
## or you can pick the variables by sorting the pvalues
para = dict(res.pvalues)
sorted_x = sorted(para.items(), key=lambda kv: kv[1])
sorted_x

[('Intercept', 5.819128944031595e-20),
 ('x[15]', 3.2163575285954144e-09),
 ('x[6]', 1.2570276234558272e-08),
 ('x[52]', 1.406533890106317e-08),
 ('x[45]', 1.4628154736541853e-06),
 ('x[24]', 3.4298885590720716e-06),
 ('x[4]', 7.941577704578332e-06),
 ('x[56]', 9.073071661167158e-06),
 ('x[51]', 1.192758743739735e-05),
 ('x[23]', 1.2318325577235966e-05),
 ('x[22]', 1.6804708331521957e-05),
 ('x[5]', 8.029454848323516e-05),
 ('x[26]', 0.00044821599032534003),
 ('x[16]', 0.00046657319670809206),
 ('x[44]', 0.0006901422214037473),
 ('x[11]', 0.0050962164499841),
 ('x[25]', 0.00675605522750552),
 ('x[35]', 0.0070309250267402736),
 ('x[7]', 0.0071886769082036),
 ('x[33]', 0.0074499395954651755),
 ('x[20]', 0.009234292859815476),
 ('x[43]', 0.009697247128588556),
 ('x[14]', 0.010907070209498822),
 ('x[48]', 0.014138691129590809),
 ('x[41]', 0.018553156266167905),
 ('x[27]', 0.03618653345875986),
 ('x[19]', 0.0394569320444539),
 ('x[47]', 0.044303550716925315),
 ('x[34]', 0.04898515409244177)

#  Use "step" function to choose a sub-model of logistic regression 
##   by AIC in a stepwise algorithm


In [16]:
import itertools
import time
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

%matplotlib inline
plt.style.use('ggplot')

In [22]:
def fit_glm_reg(X,Y):
    #Fit linear regression model and return RSS and R squared values
#     model_k = linear_model.LinearRegression(fit_intercept = True)
#     model_k.fit(X,Y) 
    indata={}
    indata['x'] = X
    indata['y'] = Y
    model_k = smf.glm(formula='y ~ x', data=indata, family=sm.families.Binomial()).fit()
#      model_k = mylogit.
    
    RSS = mean_squared_error(Y,model_k.predict()) * len(Y)
#     R_squared = model_k.score(X,Y)
# sst_val = np.sum(map(lambda x: np.power(X,2),y-np.mean(y))) 
# sse_val = np.sum(map(lambda x: np.power(X,2),m1.resid_response)) 
# r2 = 1.0 - sse_val/sst_val

    return RSS #, R_squared

In [23]:
Y = spam.loc[:, 57]
X = spam.loc[:, 0:56]

In [24]:
k = 57

remaining_features = list(X.columns.values)
features = []
# RSS_list, R_squared_list = [np.inf], [np.inf] #Due to 1 indexing of the loop...
# RSS_list = [np.inf]#Due to 1 indexing of the loop...

AIC_list = [np.inf]

features_list = dict()

for i in range(1,k+1):
#     best_RSS = np.inf
    best_AIC = np.inf
    
    for combo in itertools.combinations(remaining_features,1):

            RSS = fit_glm_reg(X[list(combo) + features],Y)   #Store temp result 
            
            #AIC
            m = len(Y)
            p = 57
            hat_sigma_squared = (1/(m - p -1)) *RSS
            AIC = (1/(m*hat_sigma_squared)) * (RSS + 2 * len(X[list(combo) + features]) * hat_sigma_squared )         
                        
            if AIC < best_AIC:
                best_AIC = AIC
#                 best_R_squared = RSS[1] 
                best_feature = combo[0]

    #Updating variables for next loop
    features.append(best_feature)
    remaining_features.remove(best_feature)
    
    #Saving values for plotting
#     RSS_list.append(best_RSS)
    AIC_list.append(best_AIC)
#     R_squared_list.append(best_R_squared)
    features_list[i] = features.copy()

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anac

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anac

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anac

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaco

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaco

...

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaco

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:889: RuntimeWarning: divide by zero encountered in true_divide
  endog_mu = self._clean(endog / mu)
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:889: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
C:\Users\tianh\Anaconda3\lib\site-packages\statsmo

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:889: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:889: RuntimeWarning: divide by zero encountered in true_divide
  endog_mu = self._clean(endog / mu)
C:\Users\tianh\Anaconda3\lib\site-packages\statsmo

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaco

C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\tianh\Anaco

In [25]:
AIC_list

[inf,
 2.9873940447728753,
 2.987394044772875,
 2.987394044772875,
 2.987394044772875,
 2.987394044772875,
 2.987394044772875,
 2.987394044772875,
 2.9873940447728753,
 2.987394044772875,
 2.9873940447728753,
 2.9873940447728753,
 2.9873940447728753,
 2.987394044772875,
 2.9873940447728753,
 2.9873940447728753,
 2.9873940447728753,
 2.9873940447728753,
 2.9873940447728753,
 2.987394044772875,
 2.987394044772875,
 2.987394044772875,
 2.9873940447728753,
 2.9873940447728753,
 2.9873940447728753,
 2.987394044772875,
 2.9873940447728753,
 2.9873940447728753,
 2.9873940447728753,
 2.987394044772875,
 2.9873940447728753,
 2.987394044772875,
 2.987394044772875,
 2.9873940447728753,
 2.987394044772875,
 2.987394044772875,
 2.987394044772875,
 2.9873940447728753,
 2.9873940447728753,
 2.987394044772875,
 2.9873940447728753,
 2.987394044772875,
 2.9873940447728753,
 2.9873940447728753,
 2.9873940447728753,
 2.9873940447728753,
 2.9873940447728753,
 2.9873940447728753,
 2.9873940447728753,
 2.987

In [ ]:
# ## Combining forward stepwise results into a new DataFrame
# df1 = pd.concat([pd.DataFrame({'features':features_list}),pd.DataFrame({'RSS':RSS_list})], axis=1, join='inner')
# df1['numb_features'] = df1.index

In [ ]:
# ## Computing the C_p, AIC, BIC 
# #Initializing useful variables
# m = len(Y)
# p = 57
# hat_sigma_squared = (1/(m - p -1)) * min(df1['RSS'])

# #Computing
# df1['C_p'] = (1/m) * (df1['RSS'] + 2 * df1['numb_features'] * hat_sigma_squared )
# df1['AIC'] = (1/(m*hat_sigma_squared)) * (df1['RSS'] + 2 * df1['numb_features'] * hat_sigma_squared )
# df1['BIC'] = (1/(m*hat_sigma_squared)) * (df1['RSS'] +  np.log(m) * df1['numb_features'] * hat_sigma_squared )
# # df1['R_squared_adj'] = 1 - ( (1 - df1['R_squared'])*(m-1)/(m-df1['numb_features'] -1))
# df1

In [ ]:
# ## Plotting the computed values as a function of number of features
# variables = ['C_p', 'AIC','BIC']
# fig = plt.figure(figsize = (18,6))

# for i,v in enumerate(variables):
#     ax = fig.add_subplot(1, 4, i+1)
#     ax.plot(df1['numb_features'],df1[v], color = 'lightblue')
#     ax.scatter(df1['numb_features'],df1[v], color = 'darkblue')
#     if v == 'R_squared_adj':
#         ax.plot(df1[v].idxmax(),df1[v].max(), marker = 'x', markersize = 20)
#     else:
#         ax.plot(df1[v].idxmin(),df1[v].min(), marker = 'x', markersize = 20)
#     ax.set_xlabel('Number of predictors')
#     ax.set_ylabel(v)

# fig.suptitle('Subset selection using C_p, AIC, BIC, Adjusted R2', fontsize = 16)
# plt.show()